In [106]:
'''
Simplex algorithm in the tableau form. The implementation should work as follows:

Take the problem input from a csv which has the following structure:

    The First line has a keyword MIN or MAX followed by a single integer d --- which is the number of variables in the problem. MIN/MAX denote whether the objective is to be minimized or maximized.
    The Second line is the unit cost vector - u a vector of length d.
    The following lines are the constraints --- one line per constraint. Each constraint of the form 'h op g^T.x'  is given in the csv file as "h op g", where 'op' is one of the comparison operators =,<=,>=; 'h' is a scalar value and g is a vector of length d. The constraints may not necessarily be independent or even consistent.
    The vectors u and g are entered in csv on a single line as a sequence (tab or comma separated) values where each 'value' is of the form "x@i" or just "x" --- "x" is any scalar value and x@i says the i-th component of the vector is x, When it is just "x" --- its position in the sequence will be taken as its index. Values at indices not in the sequence (either positional as in "x" or explicit index as in "x@i") are taken to be zero.
    You can use Numpy / JAX

The output of the Simplex implementation must be one of:

INFEASIBLE (the feasible region is empty) --- this wasn't discussed in the class. I want you to extend what was discussed in the class to detect if the given problem is infeasible.

PASS --- the optimum was successfully found

UNBOUNDED --- no optimum exists because the LP is unbounded

In addition if the output is PASS the program should print out the values of the non-zero variables as a dictionary {variable index: value} and the optimum cost at this solution along with the list of redundant constraints that were removed if any.
'''

'\nSimplex algorithm in the tableau form. The implementation should work as follows:\n\nTake the problem input from a csv which has the following structure:\n\n    The First line has a keyword MIN or MAX followed by a single integer d --- which is the number of variables in the problem. MIN/MAX denote whether the objective is to be minimized or maximized.\n    The Second line is the unit cost vector - u a vector of length d.\n    The following lines are the constraints --- one line per constraint. Each constraint of the form \'h op g^T.x\'  is given in the csv file as "h op g", where \'op\' is one of the comparison operators =,<=,>=; \'h\' is a scalar value and g is a vector of length d. The constraints may not necessarily be independent or even consistent.\n    The vectors u and g are entered in csv on a single line as a sequence (tab or comma separated) values where each \'value\' is of the form "x@i" or just "x" --- "x" is any scalar value and x@i says the i-th component of the vect

In [107]:
'''
TODO:-
=> CHECK inconsistent examples
=> Check redundant constraint examples
=> Resolve >=, <= different answer than =
'''

'\nTODO:-\n=> CHECK inconsistent examples\n=> Check redundant constraint examples\n=> Resolve >=, <= different answer than =\n'

In [108]:
import numpy as np
import pandas as pd

In [109]:
# # Represent numpy arrays as fractions
# from fractions import Fraction
# np.set_printoptions(formatter={'all':lambda x: str(Fraction(x).limit_denominator())})

In [110]:
class SimplexSolver:
    def __init__(self, type, d, u, constraints):
        self.type = type
        self.d_old = d
        self.d = d
        self.u = u
        if type == 'MAX':
            self.u = -self.u
        self.constraints = constraints
        self.addSlackVariables()
        self.G = np.array([constraint[2] for constraint in self.constraints])

        self.c = self.G.shape[0]
        self.h_tilde = np.array([constraint[0] for constraint in self.constraints])
        # self.numAuxVars = self.c - self.d

    def addSlackVariables(self):
        for i in range(len(self.constraints)):
            if self.constraints[i][1] in ['<=', '>=']:
                self.d += 1
                self.u = np.append(self.u, 0)

        # Add slack variables to the constraints according to the mask
        slack_count = 0
        for i in range(len(self.constraints)):
            if self.constraints[i][1] == '>=':
                slack_count += 1
                self.constraints[i][1] = '='                    # Convert the constraint to equality
                g_new = np.zeros(self.d)
                g_new[:self.d_old] = self.constraints[i][2]     # Copy the original constraints

                slack_index = self.d_old + slack_count - 1

                g_new[slack_index] = 1                          # Add slack variable

                self.constraints[i][2] = g_new
            elif self.constraints[i][1] == '<=':
                slack_count += 1
                self.constraints[i][1] = '='                    # Convert the constraint to equality
                g_new = np.zeros(self.d)
                g_new[:self.d_old] = self.constraints[i][2]

                slack_index = self.d_old + slack_count - 1

                g_new[slack_index] = -1

                self.constraints[i][2] = g_new
            else:
                # No need to add slack variable for equality
                self.constraints[i][2] = np.concatenate((self.constraints[i][2], np.zeros(self.d - self.d_old)))

    def getInitialLPP(self, ):
        # Get the auxiliary LPP for finding initial BFS
        # Minimize y1 + y2 + ... + yc subject to
        # [G, I] * [x, y].T = h~

        # Construct G' = [G, I]
        G1 = np.concatenate((self.G, np.eye(self.c)), axis=1)
        u1 = np.concatenate((np.zeros((self.d,)), np.ones((self.c,))))    # (d+c,)
        h_tilde1 = self.h_tilde                                    # (c,)
        return G1, u1, h_tilde1

    def getTableau(self, G, u, h_tilde):
        # Construct the tableau for the given LPP
        # Assume G: (c,d), u: (d,), h_tilde: (c,)
        uT = u.reshape(1, -1)
        tableau = np.concatenate((G, uT), axis=0)
        h_tilde = h_tilde.reshape(-1, 1)
        h_tilde = np.concatenate((h_tilde, np.zeros((1, 1))), axis=0)
        tableau = np.concatenate((tableau, h_tilde), axis=1)
        return tableau

    def getCanonicalForm(self, tableau, basis):
        '''
        Canonical form : A system Ax = b is said to be in canonical form if among
        the n variables there are m variables with the property that each appears in
        only one equation, and its coefficient in that equation is unity.
        [An introduction to optimization definition 16.5]

        To convert into canonical form, we'll make the columns corresponding
        to the basic variables in last row to 0 using row operations.
        '''
        # TODO : Can it be optimized using numpy?
        # Reduce the elements of the last row of the basic columns to 0
        for i in range(len(basis)):
            if basis[i] != -1:
                tableau[-1, :] -= tableau[basis[i], :] * tableau[-1, i]
        return tableau

    def getLeavingVar(self, tableau, entering_var, basis):
        basisMask = (basis != -1)

        # Handle precision errors
        closeZero = np.isclose(tableau, 0)
        tableau[closeZero] = 0

        # Handle unbounded LPP
        if np.all(tableau[:-1, entering_var] <= 0):
            return None
        else:
            # We need to consider only the non basic columns, so mask the basic columns
            basisMask = basisMask.reshape(1, -1)
            tableau = tableau.copy()
            # Mask will be applied on just (c,d) part of the tableau
            tableau[:-1, :-1] = tableau[:-1, :-1] * (1 - basisMask)

            # Need to find argmin(hi / nie) : nie > 0
            nie = tableau[:-1, entering_var]
            hi = tableau[:-1, -1]

            # We only need to consider the rows where nie > 0
            nMask = (nie <= 0)
            nie[nMask] = 1

            ratio = hi / nie
            ratio[nMask] = np.inf           # Set the ratio to infinity if nie = 0

            leaving_var = np.argmin(ratio)  # Returns the first occurence of the minimum value
            return leaving_var              # Note: This is the index of the leaving variable in the basis, not tableau

    def getEnteringVar(self, tableau):
        # Get the entering variable from the given tableau

        # Handle precision errors
        closeZero = np.isclose(tableau, 0)
        tableau[closeZero] = 0

        # First check if the tableau is optimal
        if np.all(tableau[-1, :-1] >= 0):
            return None
        else:
            # Get the entering variable (assuming canonical form so we're just checking entries of (u_n - N.T * u_B))
            # Find the first negative element in the last row
            entering_var = np.where(tableau[-1, :-1] < 0)[0][0]
            return entering_var

    def swapBasisVar(self, tableau, entering_var, leaving_var, basis=None):
        # Get position of leaving_var in tableau
        if basis is not None:
            leaving_var_tab = np.where(basis == leaving_var)[0][0]

        # Entering_var is the index in the tableau itself, so no need to update it

        # Update the tableau
        tableau[leaving_var, :] /= tableau[leaving_var, entering_var]
        reduce_col = tableau[:, entering_var].copy()
        reduce_col[leaving_var] = 0      # Exclude this row to prevent basis column from becoming 0
        tableau -= np.matmul(reduce_col.reshape(-1, 1), tableau[leaving_var, :].reshape(1,-1))

        if basis is not None:
            # Swap the entering and leaving variables in the basis
            basis[leaving_var_tab] = -1
            basis[entering_var] = leaving_var
            return tableau, basis
        else:
            return tableau

    def getOptimalSolution(self, tableau, basis):
        # Simplex algorithm to get the optimal solution
        entering_var = self.getEnteringVar(tableau)
        while entering_var is not None:
            leaving_var = self.getLeavingVar(tableau, entering_var, basis)
            # print("Entering var: ",entering_var)
            # print("Leaving var: ",leaving_var)
            if leaving_var is None:
                return None, None
            tableau, basis = self.swapBasisVar(tableau, entering_var, leaving_var, basis)
            entering_var = self.getEnteringVar(tableau)
            print(tableau)

        return tableau, basis

    def removeRedundantConstraints(self, ):
        # Get RREF of G matrix
        G = self.G.copy()
        pivots = []
        redundant = []
        for i in range(self.c):
            non_zero = np.where(G[i, :] != 0)[0]        # This is the pivot column
            # Constraint is redundant
            if len(non_zero) == 0:
                if (G[i, :] == 0).all():
                    redundant.append(i)
                continue
            entering_var = non_zero[0]          # Pivot column
            leaving_var = i                     # Pivot row

            # Save the pivot
            pivots.append((leaving_var, entering_var))
            G = self.swapBasisVar(G, entering_var, leaving_var)

        # print("G after RREF: \n", G)

        # If redundant constraints are present, the slack variables will consist of a basis column
        pivots = np.array(pivots)
        redundantPivots = pivots[pivots[:, 1] >= self.d_old]
        if(len(redundantPivots)):
            print("Redundant pivots: ", redundantPivots)

        redundant = np.array(redundant)
        np.concatenate((redundant, pivots[:, 0]), axis=0)
        if(len(redundant)):
            print("Redundant: " , redundant)

        # for pivot in pivots:
        #     if pivot[1] >= self.d_old:
        #         redundant.append(pivot[0])      # Redundant row
        #         print('REDUNDANT CONSTRAINTS')

        # if len(redundant) > 0:
        #     self.G = np.delete(self.G, redundant, axis=0)
        #     self.h_tilde = np.delete(self.h_tilde, redundant, axis=0)
        #     self.c -= len(redundant)

    def solve(self, ):

        # First get initial BFS by solving the auxiliary LPP
        # G1 = [G, I], u1 = [0, 0, ..d times., 0, 1, 1, ..c times., 1], h_tilde1 = h_tilde

        self.removeRedundantConstraints()

        G1, u1, h_tilde1 = self.getInitialLPP()

        tableau1 = self.getTableau(G1, u1, h_tilde1)
        # print(tableau1)

        tableau1 = self.getTableau(G1, u1, h_tilde1)

        # print(tableau1)

        basis = np.concatenate((-1 * np.ones((self.d,), dtype=int), np.arange(self.c, dtype=int)))      # To keep track of ith basic variable
        tableau = self.getCanonicalForm(tableau1, basis)
        # print(tableau)

        # Get the optimal solution
        # print(tableau.shape, basis)
        tableau, basis = self.getOptimalSolution(tableau, basis)
        # print("Tableau: \n",tableau)
        # print("Basis: ",basis)

        if tableau is None:
            if basis is None:
                return 'UNBOUNDED'

        # If [-1,-1] elem of tableau is non zero, then it is inconsistent
        if (tableau[-1,-1] != 0):
            return 'INCONSISTENT'


        # Apply simplex algorithm on the initial BFS to get the optimal solution
        # Remove the columns corresponding to the auxiliary variables

        # Remove last c columns and add the last column to the end
        tableau = np.concatenate((tableau[:-1, :-(self.c+1)], tableau[:-1, -1].reshape(-1,1)), axis=1)
        u_0 = np.concatenate((self.u.reshape(1,-1), np.zeros((1,1))), axis=1)
        tableau = np.concatenate((tableau, u_0), axis=0)

        # New basis
        basis = basis[:-self.c]
        # print(basis.shape)

        # Get canonical form
        print(tableau)
        tableau = self.getCanonicalForm(tableau, basis)
        print(tableau)

        # Optimize the new tableau
        tableau, basis = self.getOptimalSolution(tableau, basis)
        print("Tableau: ",tableau)
        print("Basis: ",basis)

        # LPP is unbounded
        if tableau is None:
            if basis is None:
                return 'UNBOUNDED'

        # Print the final tableau
        print(tableau)
        return "PASS"

In [111]:
# type = "MIN"          # fun1 = 0  fun2 = 0 INCOSISTENT
# d = 2
# u = np.array([2, 3])
# constraints = [
#     [12, "=", np.array([4, 2])],
#     [6, "=", np.array([1, 4])],
#     [7, "=", np.array([1, 4])],
# ]

# type = "MAX"            # fun1 = -6.4  fun2 = -2 PASS VAL = 6.4
# d = 3
# u = np.array([4, 1, 4])
# constraints = [
#     [2, ">=", np.array([2,1,1])],
#     [4, ">=", np.array([1,2,3])],
#     [8, ">=", np.array([2,2,1])],
# ]

# type="MIN"          # fun1 = 0  fun2 = 0 PASS VAL = 0
# d=4
# u=np.array([1,1,0,0])
# constraints=[
#     [8, "=", np.array([1,1,0,1])],
#     [10, "=", np.array([2, 1, 1, 0])],
# ]

# type = "MIN"        # fun1 = 0  fun2 = 0 INCONSISTENT
# d=2
# u = np.array([5, 7])
# constraints = [
#     [5, ">=", np.array([1, 1])],
#     [6, "<=", np.array([1, 1])],
# ]

# type="MIN"          # fun1 = 0  fun2 = 3.6 PASS VAL = -3.6
# d=2
# u=np.array([4,1])
# constraints = [
#     [3, ">=", np.array([1, 2])],
#     [6, "<=", np.array([4, 3])],
#     [3, "=", np.array([3, 1])],
# ]

# type="MIN"          # fun1 = 0.9999999999999998  fun2 = 0.9999999999999998 PASS VAL = -1
# d=4
# u=np.array([1,-1,2,-2])
# constraints = [
#     [1, "=", np.array([1,-1,1,-2])],
#     [4, "=", np.array([2,-2,1,-1])],
# ]

# type="MAX"          # fun1 = -13.3333333333333  fun2 = -13.3333333333333 PASS VAL = 20 , pehle 13.3333333333333 aata hai , phir 20 ho jata hai
# d=2
# u=np.array([3,5])
# constraints = [
#     [4, ">=", np.array([1,1])],
#     [8, "<=", np.array([5,3])],
# ]

# type="MIN"          # fun1 = -5  fun2 = -1 PASS VAL = 1
# d=4
# u=np.array([2,-1,-1,0])       # works
# constraints = [
#     [4, "=", np.array([3,1,0,1])],
#     [5, "=", np.array([6,2,1,1])],
# ]

# type="MAX"          # fun1 = -4  fun2 = -4 PASS VAL = 4
# d=3
# u=np.array([1,1,3])    # works
# constraints = [
#     [1, "=", np.array([1,0,1])],
#     [2, "=", np.array([0,1,1])],
# ]

# type="MAX"          # fun1 = -17 fun2 = -17 PASS VAL = 17
# d=2
# u=np.array([2,1])       # works
# constraints = [
#     [5, ">=", np.array([1,0])],
#     [7, ">=", np.array([0,1])],
# ]

# type="MIN"        # fun1 = 0 fun2 = 2 PASS VAL = -2
# d=2
# u=np.array([1,1])       # works
# constraints = [
#     [3, "<=", np.array([1,2])],
#     [3, "<=", np.array([2,1])],
# ]


# type="MAX"          # fun1 = -113.33333333 fun2 = -110.52083333333333 PASS VAL = 113.33333333
# d=4
# u=np.array([6,4,7,5])       # works
# constraints = [
#     [20, ">=", np.array([1,2,1,2])],
#     [100, '>=', np.array([6,5,3,2])],
#     [75, ">=", np.array([3,4,9,12])],
# ]

# for INCONSISTENT
# type = "MIN"            # fun1 = 0  fun2 = 0 INCONSISTENT
# d=2
# u = np.array([5, 7])
# constraints = [
#     [5, ">=", np.array([1, 1])],
#     [6, "<=", np.array([1, 1])],
# ]

# for UNBOUNDED
# type = "MAX"            # fun1 = -12  fun2 = -12 UNBOUNDED
# d=2
# u = np.array([2, 2])
# constraints = [
#     [6, "<=", np.array([1, 1])]
# ]

# type="MAX"          # fun1 = 14.66666666666 fun2 = 0 PASS VAL = -14.66666666666
# d=2
# u=np.array([-4,-3])       # works
# constraints = [
#     [11, "<=", np.array([5,1])],
#     [-8, ">=", np.array([-2,-1])],
#     [7. , "<=", np.array([1,2])],
# ]

# type = "MIN"
# d = 3
# u = np.array([1, 2, 3])
# constraints = [
#     [2, "=", np.array([-2, 4, 7])],
#     [3, "=", np.array([0, 1, 2])],
# ]

# type = "MIN"          # WRONG ANSWER [1) Giving redundant constraint but not actually redundant, 2) Not following the constraint even if wrong redundancy not removed]
# d = 3
# u = np.array([1, 2, 3])
# constraints = [
#     [1, "<=", np.array([1, 2, 3])],
#     [2, ">=", np.array([-2, 4, 7])],
#     [3, ">=", np.array([0, 1, 2])],
#     [3, "<=", np.array([0, 1, 2])]
# ]

# type = "MIN"          # WRONG ANSWER [1) Giving redundant constraint but not actually redundant, 2) Not following the constraint even if wrong redundancy not removed]
# d = 3
# u = np.array([1, 2, 3])
# constraints = [
#     [1, "<=", np.array([1, 2, 3])],
#     [7, ">=", np.array([1, 0, 1])],
#     [5, "<=", np.array([1, 0, 1])]
# ]

# type = "MIN"          # WORKING
# d = 3
# u = np.array([1, 2, 3])
# constraints = [
#     [1, "<=", np.array([1, 2, 3])],
#     [2, ">=", np.array([-2, 4, 7])],
#     [3, "=", np.array([0, 1, 2])],
#     [3, "=", np.array([0, 1, 2])]
# ]

# type="MIN"
# d=2
# u=np.array([1,2])
# constraints = [
#     [6, "=", np.array([1,2])],
#     [5, "=", np.array([1,1])],
#     [8, "=", np.array([2,3])],
# ]

# type = "MAX"            # fun1 = -4  fun2 = -4 PASS VAL = 4
# d = 3
# u = np.array([1, 1, 3])
# constraints = [
#     [1, "= ", np.array([1, 0, 1])],
#     [2, "=", np.array([0, 1, 1])],
# ]

# type = "MAX"
# d = 2
# u = np.array([3,5])
# constraints = [
#     [4, ">=", np.array([1,1])],
#     [8, "<=", np.array([5,3])],
# ]


In [112]:
import pandas as pd
import numpy as np
def take_input():
    input_file = pd.read_csv('input.csv', header=None)
    # print(input_file)
    type = input_file.iloc[0,0]
    MAX_MIN = type.split(' ')[0]
    d = int(type.split(' ')[1])
    u = list(map(float, input_file.iloc[1, :].tolist()[0].split()))
    u = np.array(u)
    constraints = []
    for i in range(2, len(input_file)):
        curr_row = input_file.iloc[i, :].tolist()[0]
        split_row = [curr_row.split()[0], curr_row.split()[1], ' '.join(curr_row.split()[2:])]

        _b_ = float(split_row[0])
        _ineq_ = split_row[1]

        val_A = split_row[2].split()
        # if val_A has '@' in it then it means we make a vector of length d
        # vector will have 0 at all places except the index mentioned in val_A
        # the second value is an index of the vector where we need to put the value
        if '@' in val_A[0]:
            _A_ = np.zeros(d)
            for val in val_A:
                val = val.split('@')
                _A_[int(val[1])] = val[0]

        else:
            print("val_A: ",val_A)
            _A_ = np.array(list(map(float, split_row[2].split())))

        joint_row = [_b_, _ineq_, _A_]

        constraints.append(joint_row)
    return MAX_MIN, d, u, constraints

In [113]:
# csv file waale ka bhi fun1 = -4 fun2 = -4 PASS VAL = 4
type, d, u, constraints = take_input()

print("type: ",type)
print("d: ",d)
print("u: ",u)
print("constraints: ",constraints)

simplex = SimplexSolver(type, d, u, constraints)
print()
print("type: ",type)
print("d: ",d)
print("u: ",u)
print("constraints: ",constraints)

val_A:  ['1', '2', '4']
val_A:  ['1', '3', '7']
type:  MAX
d:  3
u:  [2. 5. 4.]
constraints:  [[4.0, '>=', array([1., 2., 4.])], [6.0, '>=', array([ 0.,  0., 10.])], [8.0, '>=', array([1., 3., 7.])]]

type:  MAX
d:  3
u:  [2. 5. 4.]
constraints:  [[4.0, '=', array([1., 2., 4., 1., 0., 0.])], [6.0, '=', array([ 0.,  0., 10.,  0.,  1.,  0.])], [8.0, '=', array([1., 3., 7., 0., 0., 1.])]]


In [114]:
ans = simplex.solve()
print("ans: ",ans)

[[  1.   2.   4.   1.   0.   0.   1.   0.   0.   4.]
 [  0.   0.  10.   0.   1.   0.   0.   1.   0.   6.]
 [  0.   1.   3.  -1.   0.   1.  -1.   0.   1.   4.]
 [  0.  -1. -13.   1.  -1.  -1.   2.   0.   0. -10.]]
[[  0.5   1.    2.    0.5   0.    0.    0.5   0.    0.    2. ]
 [  0.    0.   10.    0.    1.    0.    0.    1.    0.    6. ]
 [ -0.5   0.    1.   -1.5   0.    1.   -1.5   0.    1.    2. ]
 [  0.5   0.  -11.    1.5  -1.   -1.    2.5   0.    0.   -8. ]]
[[ 0.5  1.   0.   0.5 -0.2  0.   0.5 -0.2  0.   0.8]
 [ 0.   0.   1.   0.   0.1  0.   0.   0.1  0.   0.6]
 [-0.5  0.   0.  -1.5 -0.1  1.  -1.5 -0.1  1.   1.4]
 [ 0.5  0.   0.   1.5  0.1 -1.   2.5  1.1  0.  -1.4]]
[[ 0.5  1.   0.   0.5 -0.2  0.   0.5 -0.2  0.   0.8]
 [ 0.   0.   1.   0.   0.1  0.   0.   0.1  0.   0.6]
 [-0.5  0.   0.  -1.5 -0.1  1.  -1.5 -0.1  1.   1.4]
 [ 0.   0.   0.   0.   0.   0.   1.   1.   1.   0. ]]
[[ 0.5  1.   0.   0.5 -0.2  0.   0.8]
 [ 0.   0.   1.   0.   0.1  0.   0.6]
 [-0.5  0.   0.  -1.5 -0.1  1.  

In [115]:
a=1/13
c=4/13
print(a+2*c)
print(2*c)
print()

0.6923076923076923
0.6153846153846154



In [116]:
# Create a function to transform from constraints to G for linprog
def constraintsToG(constraints):
    G = np.zeros((len(constraints), d))
    for i in range(len(constraints)):
        print(constraints[i])
        if constraints[i][1] == '<=':
            G[i] = -1 * constraints[i][2][:d]
        elif constraints[i][1] == '>=':
            G[i] = constraints[i][2][:d]
        elif constraints[i][1] == '=':
            # TODO : Change = to >= and <= constraints
            G[i] = constraints[i][2][:d]
            # G = np.concatenate((G, -1 * constraints[i][2][:d].reshape(1,-1)), axis=0)
    return G

def constraintsToH(constraints):
    h = np.zeros((len(constraints),))
    for i in range(len(constraints)):
        if constraints[i][1] == '<=':
            h[i] = -1 * constraints[i][0]
        elif constraints[i][1] == '>=':
            h[i] = constraints[i][0]
        elif constraints[i][1] == '=':
            h[i] = constraints[i][0]
            # h = np.concatenate((h, -1 * constraints[i][0]), axis=0)
    return h

In [117]:
# Solve the LPP using linprog
from scipy.optimize import linprog
c = simplex.u[:simplex.d_old]
print(c)
G = constraintsToG(constraints)
print(G)
h = constraintsToH(constraints)       # Form h >= g^T.x
print(h)
# Print with steps
res = linprog(c, A_ub=G, b_ub=h, method='simplex', options={'disp': True})
print(res)

[-2. -5. -4.]
[4.0, '=', array([1., 2., 4., 1., 0., 0.])]
[6.0, '=', array([ 0.,  0., 10.,  0.,  1.,  0.])]
[8.0, '=', array([1., 3., 7., 0., 0., 1.])]
[[ 1.  2.  4.]
 [ 0.  0. 10.]
 [ 1.  3.  7.]]
[4. 6. 8.]
Optimization terminated successfully.
         Current function value: -10.000000  
         Iterations: 4
 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -10.0
       x: [ 0.000e+00  2.000e+00  0.000e+00]
     nit: 4


C:\Users\offic\AppData\Local\Temp\ipykernel_3584\399770180.py:10: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res = linprog(c, A_ub=G, b_ub=h, method='simplex', options={'disp': True})


In [118]:
# Solve the LPP using linprog
from scipy.optimize import linprog
c = simplex.u[:simplex.d_old]
print(c)
G = constraintsToG(constraints)
print(G)
h = constraintsToH(constraints)       # Form h >= g^T.x
print(h)
# Print with steps
res = linprog(c, A_eq=G, b_eq=h, method='simplex', options={'disp': True})
print(res)

[-2. -5. -4.]
[4.0, '=', array([1., 2., 4., 1., 0., 0.])]
[6.0, '=', array([ 0.,  0., 10.,  0.,  1.,  0.])]
[8.0, '=', array([1., 3., 7., 0., 0., 1.])]
[[ 1.  2.  4.]
 [ 0.  0. 10.]
 [ 1.  3.  7.]]
[4. 6. 8.]
Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 1.4e+00 which exceeds the required tolerance of 1e-09 for a solution to be considered 'close enough' to zero to be a basic solution. Consider increasing the tolerance to be greater than 1.4e+00. If this tolerance is unacceptably  large the problem may be infeasible.
         Iterations: 1
 message: Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 1.4e+00 which exceeds the required tolerance of 1e-09 for a solution to be considered 'close enough' to zero to be a basic solution. Consider increasing the tolerance to be greater than 1.4e+00. If this tolerance is unacceptably  large the problem may be infeasible.
 succe

C:\Users\offic\AppData\Local\Temp\ipykernel_3584\4065296701.py:10: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res = linprog(c, A_eq=G, b_eq=h, method='simplex', options={'disp': True})
